In [48]:
import pandas as pd
import numpy as np
from clustering_alg import custom_clustering
import gensim.models
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer

In [17]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')
df_train.head()

,Id,Comment,Topic
0,0x840,A few things. You might have negative- frequen...,Biology
1,0xbf0,Is it so hard to believe that there exist part...,Physics
2,0x1dfc,There are bees,Biology
3,0xc7e,I'm a medication technician. And that's alot o...,Biology
4,0xbba,Cesium is such a pretty metal.,Chemistry


In [38]:
df_train.shape

(8695, 3)

In [34]:
for comment in df_train.Comment:
    df_train.loc[df_train['Comment']==comment] = comment.replace("\n"," ")
    df_train.loc[df_train['Comment']==comment] = comment.replace("\\n"," ")
df_train.iloc[0,1]

'A few things. You might have negative- frequency dependent selection going on where the least common phenotype, reflected by genotype, is going to have an advantage in the environment. For instance, if a prey animal such as a vole were to have a light and a dark phenotype, a predator might recognize the more common phenotype as food.  So if the light voles are more common, foxes may be keeping a closer eye out for light phenotypic voles, recognising them as good prey. This would reduce the light causing alleles due to increased predation and the dark genotypes would increase their proportion of the population until this scenario is reversed. This cycle continues perpetually.   However, this is unlikely to be strictly yearly as it usually takes more time than a year for an entire populations allele frequencies to change enough to make a large enough difference to alter fitness.   More likely on a *year to year* basis, the population is experiencing fluctuating selection where alternati

In [41]:
comments_list = df_train.Comment.tolist()
w2v_data = list()
for comment in comments_list:
    for sentence in sent_tokenize(comment):
        temp = []
        for word in word_tokenize(sentence):
            temp.append(word.lower())
    w2v_data.append(temp)    
w2v_data[0]


['if',
 'there',
 'is',
 'no',
 'flooding',
 ',',
 'the',
 'dry-type',
 'genotype',
 'will',
 'have',
 'more',
 'fitness',
 'leading',
 'to',
 'more',
 'offspring',
 'and',
 'therefore',
 'more',
 'dry',
 'alleles',
 'in',
 'the',
 'population',
 ',',
 'however',
 ',',
 'in',
 'flooded',
 'years',
 'the',
 'wet-liking',
 'phenotype',
 'will',
 'do',
 'better',
 'and',
 'propagate',
 'the',
 'wet',
 'genes',
 '.']

In [74]:
model = gensim.models.Word2Vec(w2v_data, min_count = 1, vector_size = 100, window = 5)
model.wv.similarity('fitness','biology')
# print("fitness and biology similarity: "+str(model.wv.similarity('fitness','biology')))
# print("fitness and chemistry similarity: "+str(model.wv.similarity('fitness','chemistry')))

0.77478534